In [5]:
pip install holidays

  Using cached holidays-0.11.3.1-py3-none-any.whl (155 kB)
  Using cached korean_lunar_calendar-0.2.1-py3-none-any.whl (8.0 kB)
  Using cached convertdate-2.3.2-py3-none-any.whl (47 kB)
  Using cached hijri_converter-2.2.2-py3-none-any.whl (13 kB)
  Using cached PyMeeus-0.5.11.tar.gz (5.4 MB)
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730977 sha256=a3ff72174747d9d076bbdf4524d79bab4d91d532d9ad2ecedba8f7d7b1531bff
  Stored in directory: c:\users\devik\appdata\local\pip\cache\wheels\a0\8b\b2\810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import seaborn as sns

import datetime
import ast
import holidays

In [7]:
def load_dataset():
  df = pd.read_csv('data/movies_metadata.csv',dtype='unicode')
  return df


Columns to be dropped:
1. belongs_to_collection
2. homepage
3. Overview (hume NLP nahi aati :/)
4. poster_path (hume CV nahi aati :/)
5. Status 
6. release_date (kya hi kar lenge iska)
7. tagline (hume NLP nahi aati :/)
8. Video (kya hi kar lenge iska)
9. Spoken Languages
10. original_language ("en","fr", "it","ja","de") (????)
11. adult (T/F)

Counting:
1. budget (Integer)
=> Remove 0 and null values
2. Genres (Dictionary {id,genre_name})
3. movie_id (Unique int)
=> required for joining with other features not in this database
4. imdb_id (Unique string)
5. original title 
6. popularity (float)
7. Production Companies
8. Production Country (?????)
9. Revenue 
=> remove samples with 0 and null
10. runtime
11. Title
12. Vote average
13. Vote count


In [8]:
genre_set = {'Family', 'Animation', 'Thriller', 'Documentary', 'Music', 'Horror', 'Foreign', 'Western', 'TV Movie',
           'Fantasy', 'Mystery', 'Action', 'Romance', 'History', 'Comedy', 'Adventure', 'War', 'Science Fiction', 'Crime', 'Drama'}

def drop_columns(df):
  df.drop(["adult",
           "belongs_to_collection",
           "homepage",
           "spoken_languages", 
           "original_title",
           "overview",
           "poster_path",
           "vote_average",
           "production_companies",
           "status",
           "tagline", 
           "title",
           "video",
           "vote_count"],axis = 1, inplace = True)
  return df


def removeNa(movies):
  movies.dropna(subset=['budget', 'genres', 'id', 'imdb_id','popularity', 
                        "production_countries", "release_date",
                        "revenue" , "runtime", "original_language"], inplace = True)
  movies.reset_index(drop = True, inplace = True)
  return movies

def drop_zero_revenue_samples(df):
  zero_rows = df[ df['revenue'] == '0' ].index
  df = df.drop(zero_rows)
  return df

def drop_zero_budget_samples(df):
  zero_rows = df[ df['budget'] == '0' ].index
  df = df.drop(zero_rows)
  return df

def drop_zero_p_com_samples(df):
  zero_rows = df[ df['production_companies'] == '[]' ].index
  df=df.drop(zero_rows)
  return df

def drop_zero_p_con_samples(df):
  zero_rows = df[ df['production_countries'] == '[]' ].index
  df=df.drop(zero_rows)
  return df

def remove_zeros(df):
  df = drop_zero_revenue_samples(df)
  df = drop_zero_budget_samples(df)
  df = df.reset_index(drop=True)
  return df

# function to convert data types of different columns
def convert_datatype(movies):
  datatype_dict = {"budget": np.int64,
                   "id": np.int64, 
                   "popularity": np.float64, 
                   "revenue": np.int64,
                   "runtime": np.float64}
  movies = movies.astype(datatype_dict, copy=True)
  return movies

def remove_revenue_outliers(df):
  df=df[df['revenue']>1000]
  return df

# function to call the above functions in order
def preprocess_dataset(df):
  df = drop_columns(df)
  df = removeNa(df)
  df = remove_zeros(df)
  df = convert_datatype(df)
  print(df.shape)
  df = remove_revenue_outliers(df)
  df = df.reset_index(drop=True)
  print(df.shape)
  return df

# Preprocessing
1. Removing unwanted features
2. Removing 0/Null '[ ]' values from remaining samples
3. Conversion of datatypes from object to float/int

In [9]:
df = load_dataset()
df = preprocess_dataset(df)

(5380, 10)
(5320, 10)


In [10]:
df

,budget,genres,id,imdb_id,original_language,popularity,production_countries,release_date,revenue,runtime
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0
2,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0
3,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949,tt0113277,en,17.924927,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818,170.0
4,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",9091,tt0114576,en,5.231580,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,64350171,106.0
...,...,...,...,...,...,...,...,...,...,...
5315,11000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",395834,tt5362988,en,40.796775,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2017-08-03,184770205,111.0
5316,12000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",24049,tt0479751,ta,1.323587,"[{'iso_3166_1': 'IN', 'name': 'India'}]",2007-06-14,19000000,185.0
5317,800000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",62757,tt0933361,en,0.903061,[],2006-11-23,1328612,100.0
5318,2000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",63281,tt1718881,en,0.121844,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",2010-09-30,1268793,107.0


#Process Keywords
1. Get a dictionary of all keywords and their counts.
2. Take top N keywords.
3. Count number of top N keywords present in every sample from the dataset.
4. Add the above count as a column to the main dataset.


In [13]:
# After we have the final df post preprocessing, we just add the columns
def convert_datatype_keywords(keyword_df):
  keyword_df = keyword_df.astype({"id": np.int64}, copy=True)
  keyword_df['keywords'] = keyword_df['keywords'].apply(lambda row : ast.literal_eval(str(row)))
  return keyword_df

def get_all_keyword_dictionary(df):
  all_keywords={}
  for a_list in df['keywords']:
    if a_list!=[]:
      for string in a_list:
        indv_dict = ast.literal_eval(str(string))
        keyword = indv_dict["name"]
        if keyword in all_keywords:
          curr=all_keywords[keyword]
          curr+=1
          all_keywords[keyword]=curr
        else:
          all_keywords[keyword]=1
  return all_keywords

def keep_max_n_values(all_keywords,n):
  all_keywords=dict(sorted(all_keywords.items(), key=lambda item: item[1],reverse=True)[:n])
  return list(all_keywords.keys())

def get_counts_keywords(keyword_df,top_n_list):
  new_column=[]
  for a_list in keyword_df['keywords']:
    count=0
    if a_list!=[]:
      for string in a_list:
        indv_dict = ast.literal_eval(str(string))
        keyword = indv_dict["name"]
        if keyword in top_n_list:
          count+=1
    new_column.append(count)
  return new_column

def get_and_process_keywords(current_ids):
  '''
  1. Get a dictionary of all keywords and their counts.
  2. Take top N keywords.
  3. Count number of top N keywords present in every sample from the dataset.
  4. Add the above count as a column to the main dataset.
  '''
  keyword_df = pd.read_csv('data/keywords.csv', dtype='unicode')

  keyword_df = convert_datatype_keywords(keyword_df)
  keyword_df = keyword_df[keyword_df['id'].isin(current_ids)]
  keyword_df.drop_duplicates(subset=['id'],inplace=True)
  keyword_df = keyword_df.reset_index(drop=True)
  #1.
  all_keywords=get_all_keyword_dictionary(keyword_df)
  #2.
  top_n_list=keep_max_n_values(all_keywords,1500)
  #3.
  count_column= get_counts_keywords(keyword_df,top_n_list)
  keyword_df['keyword'] = count_column
  keyword_df.drop(["keywords"],axis=1,inplace=True)
  keyword_df = keyword_df.reset_index(drop=True)
  
  return keyword_df

def merge_main_and_keyword(main_df,keyword_df):
  new_df=pd.merge(main_df, keyword_df, on="id")
  return new_df

def process_keywords(df):
  current_ids = df['id'].tolist()
  keyword_df = get_and_process_keywords(current_ids)
  df = merge_main_and_keyword(df,keyword_df)
  return df

df = process_keywords(df)

In [14]:
# Dataset after processing keywords
df

,budget,genres,id,imdb_id,original_language,popularity,production_countries,release_date,revenue,runtime,keyword
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,7
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,2
2,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,4
3,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949,tt0113277,en,17.924927,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818,170.0,17
4,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",9091,tt0114576,en,5.231580,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,64350171,106.0,3
...,...,...,...,...,...,...,...,...,...,...,...
5315,11000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",395834,tt5362988,en,40.796775,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2017-08-03,184770205,111.0,10
5316,12000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",24049,tt0479751,ta,1.323587,"[{'iso_3166_1': 'IN', 'name': 'India'}]",2007-06-14,19000000,185.0,0
5317,800000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",62757,tt0933361,en,0.903061,[],2006-11-23,1328612,100.0,0
5318,2000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",63281,tt1718881,en,0.121844,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",2010-09-30,1268793,107.0,0


#Processing Date

1. Segregrate year of release from the release dates.
2. Separate days (monday = 0, tuesday = 1 ... ) from the release dates.

In [15]:
def extract_year(df):
  """
  function to extract year from release date and append to `df`
  """
  df['release_date'] = pd.to_datetime(df['release_date'],format='%Y-%m-%d')
  df['Year'] = pd.DatetimeIndex(df['release_date']).year
  return df  
  

def extract_dow(df):
  """
  function to find day from release date and append to `df`
  """
  df_days = []
  for i in range(len(df)):
    date = df.iloc[i].release_date
    day = datetime.datetime.strptime(date, '%Y-%m-%d').weekday()
    df_days.append(day)
  df['Day'] = df_days
  return df

# function to call above functions
def process_date(df):
  df = extract_dow(df)
  df = extract_year(df)
  return df

df = process_date(df)

#Processing Genres
1. One hot encode the genres

In [16]:
# Separating the genre

def column_management(df, genre_name):
  curr_cols = df.columns.values
  genre_list = list(genre_name)
  new_columns = np.append(curr_cols[:-20], genre_list)
  df.columns = new_columns
  return df

def get_genre_map(genre_set):
  genre_map={}
  ind=0
  for genres in genre_set:
      genre_map[genres]=ind
      ind+=1
  return genre_map

def genre_process(df,genre_set):
  n=df.shape[0]
  genre_map=get_genre_map(genre_set)
  dummy=np.zeros((n,20))
  count=0
  for samples in df['genres']:
    val = ast.literal_eval(samples)
    if(val!=[]):
      for string in val:
        res = ast.literal_eval(str(string))
        genre = res["name"]
        dummy[count,genre_map[genre]]=1
    count+=1
  df.drop(["genres"],axis=1,inplace=True)
  df_new = pd.concat([df, pd.DataFrame(dummy)], axis=1)
  df=column_management(df_new,genre_set)
  return df


df = genre_process(df,genre_set)

In [17]:
df

,budget,id,imdb_id,original_language,popularity,production_countries,release_date,revenue,runtime,keyword,...,Action,Drama,Comedy,Foreign,Horror,Adventure,Music,Animation,Thriller,Documentary
0,30000000,862,tt0114709,en,21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,7,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,65000000,8844,tt0113497,en,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249,104.0,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,16000000,31357,tt0114885,en,3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156,127.0,4,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,60000000,949,tt0113277,en,17.924927,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818,170.0,17,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,35000000,9091,tt0114576,en,5.231580,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,64350171,106.0,3,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,11000000,395834,tt5362988,en,40.796775,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2017-08-03,184770205,111.0,10,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5316,12000000,24049,tt0479751,ta,1.323587,"[{'iso_3166_1': 'IN', 'name': 'India'}]",2007-06-14,19000000,185.0,0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5317,800000,62757,tt0933361,en,0.903061,[],2006-11-23,1328612,100.0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5318,2000000,63281,tt1718881,en,0.121844,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",2010-09-30,1268793,107.0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Processing Date

one hot encode release dates based on if they are released during a holiday, near a holiday or a non-holiday day

In [18]:
# Do an exploratory data analysis on holiday

def eda_holidays():
  country_holidays = holidays.USA() + holidays.UK()

  M = df.shape[0]
  count = 0 # Total number of holidays
  for i in range(M):
    if country_holidays.get(df.iloc[i]['release_date']) != None:
      count += 1
  print(f"Total number of holidays = {count}\n")

  map = {}

  for i in range(M):
    holiday = country_holidays.get(df.iloc[i]['release_date'])
    if holiday != None:
      if holiday in map:
        map[holiday] += 1
      else:
        map[holiday] = 1

  for itr in map:
    print(f"{itr}: {map[itr]}")

eda_holidays()

Total number of holidays = 253

Veterans Day (Observed): 3
Veterans Day: 15
St. Patrick's Day [Northern Ireland]: 18
Christmas Day: 64
Battle of the Boyne [Northern Ireland]: 13
Good Friday: 27
Independence Day: 12
Late Summer Bank Holiday [England/Wales/Northern Ireland]: 1
Washington's Birthday: 5
Thanksgiving: 11
New Year's Day: 33
Boxing Day: 11
St. Andrew's Day [Scotland]: 3
Martin Luther King Jr. Day: 3
Armistice Day: 1
Easter Monday [England/Wales/Northern Ireland]: 3
May Day: 3
Labor Day: 14
Columbus Day: 2
Memorial Day, Spring Bank Holiday: 2
New Year Holiday [Scotland]: 3
Memorial Day: 1
Christmas Day (Observed): 1
Wedding of William and Catherine: 2
Summer Bank Holiday [Scotland]: 2


In [19]:
def preprocess_holidays():
  """
  function to pre-process holidays
  """

  country_holidays = holidays.USA() + holidays.UK()

  M = df.shape[0]

  is_holiday = []
  is_near_holiday = []

  # for all the samples in dataset check if it's a holiday or near some holiday
  for i in range(M):
    
    holiday = country_holidays.get(df.iloc[i]['release_date'])

    # if it's a holiday append 1 and we're done
    if holiday != None:
      is_holiday.append(1)
      is_near_holiday.append(1)
    
    else:
      is_holiday.append(0)
      flag = False

      # if it's not a holiday then check if any of the nearby (+-2) days were a holiday
      for j in range(-2,3):
        # find the date j days ahead 
        date = df.iloc[i]['release_date'] + datetime.timedelta(days = j)
        # check if `date` was a holiday
        holiday = country_holidays.get(date)
        if holiday != None:
          flag = True
          break
        
      if flag:
        is_near_holiday.append(1)
      else:
        is_near_holiday.append(0)

  return df.assign(is_holiday=is_holiday, is_near_holiday=is_near_holiday)

df = preprocess_holidays()

# Language
Since 90% of the movies are in English, one hot encode.

In [20]:
def process_language(df):
  # Changing datatype of column to string
  df['original_language'] = df['original_language'].astype("string")

  n = df.shape[0]
  language = np.zeros(n)
  for i in range(n):
    if df.loc[i].at['original_language'] == 'en':
      language[i] = 1
    
  language = pd.DataFrame(language)
  df['original_language'] = language
  df.rename(columns = {'original_language': 'language'}, inplace = True)

  return df

df = process_language(df)

# Country
Out of 80 unique production companies, 8 had > 100 entries. So creating binary columns for each and one for other country too.

In [21]:
def getCountryInfo(df):
  # Getting the count
  d = {}
  n = df.shape[0] # number of entries
  for i in range(n):
    sample = df.loc[i].at['production_countries']
    val = ast.literal_eval(sample) # convert from string to list
    if(val != []):
      for string in val:
        res = ast.literal_eval(str(string))
        c = res["name"]
        if c in d:
          d[c] += 1
        else:
          d[c] = 1
  # print(sorted(d.items(), key=lambda x: x[1], reverse=True))
  # Output - top 8 countries have > 100 entries
  # Creating a binary column for each of those and one for others

country_set = {'United States of America', 'United Kingdom', 'France', 'Germany', 'Canada', 'India', 'Australia', 'Italy', 'Other_Country'}

def column_management(df, country_name):
  curr_cols = df.columns.values
  country_list = list(country_name)
  m = df.shape[1]
  m -= 9
  new_columns = np.append(curr_cols[:m], country_list)
  df.columns = new_columns
  return df

def process_country(df, country_set):
  n = df.shape[0]
  country_map = {}
  ind = 0
  for countries in country_set:
      country_map[countries] = ind
      ind += 1

  dummy = np.zeros((n, 9))
  count = 0
  for samples in df['production_countries']:
    val = ast.literal_eval(samples)
    if(val != []):
      for string in val:
        res = ast.literal_eval(str(string))
        c = res["name"]
        if c in country_map:
          dummy[count, country_map[c]] = 1
        else:
          dummy[count, country_map['Other_Country']] = 1 # Other_Country
    count += 1

  df.drop(["production_countries"], axis = 1, inplace = True)
  df_new = pd.concat([df, pd.DataFrame(dummy)], axis = 1)
  df = column_management(df_new, country_set)
  return df

getCountryInfo(df)
df = process_country(df, country_set)
df

,budget,id,imdb_id,language,popularity,release_date,revenue,runtime,keyword,Day,...,is_near_holiday,Germany,Australia,India,Canada,United Kingdom,Other_Country,France,United States of America,Italy
0,30000000,862,tt0114709,1.0,21.946943,1995-10-30,373554033,81.0,7,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,65000000,8844,tt0113497,1.0,17.015539,1995-12-15,262797249,104.0,2,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,16000000,31357,tt0114885,1.0,3.859495,1995-12-22,81452156,127.0,4,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,60000000,949,tt0113277,1.0,17.924927,1995-12-15,187436818,170.0,17,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,35000000,9091,tt0114576,1.0,5.231580,1995-12-22,64350171,106.0,3,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,11000000,395834,tt5362988,1.0,40.796775,2017-08-03,184770205,111.0,10,3,...,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
5316,12000000,24049,tt0479751,0.0,1.323587,2007-06-14,19000000,185.0,0,3,...,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5317,800000,62757,tt0933361,1.0,0.903061,2006-11-23,1328612,100.0,0,3,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5318,2000000,63281,tt1718881,1.0,0.121844,2010-09-30,1268793,107.0,0,3,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


#Processing Cast and Crew
1. Get a dictionary of all cast and crew and their counts
2. Take top n cast and crew
3. Parse every sample and check how many of the top n are present in it
4. Finally add columns to main database

In [23]:
def convert_datatype_cast(credit_df):
  credit_df = credit_df.astype({"id": np.int64}, copy=True)
  credit_df['cast'] = credit_df['cast'].apply(lambda row : ast.literal_eval(str(row)))
  credit_df['crew'] = credit_df['crew'].apply(lambda row : ast.literal_eval(str(row)))
  return credit_df

def get_all_cast_dictionary(df,label):
  all_credits={}
  for a_list in df[label]:
    if a_list!=[]:
      for string in a_list:
        indv_dict = ast.literal_eval(str(string))
        credit = indv_dict["name"]
        if credit in all_credits:
          curr=all_credits[credit]
          curr+=1
          all_credits[credit]=curr
        else:
          all_credits[credit]=1
  return all_credits


def get_counts_cast(credit_df,top_n_list,label):
  new_column=[]
  for a_list in credit_df[label]:
    count=0
    if a_list!=[]:
      for string in a_list:
        indv_dict = ast.literal_eval(str(string))
        credit = indv_dict["name"]
        if credit in top_n_list:
          count+=1
    new_column.append(count)
  return new_column

def get_and_process_credits(current_ids):
  credit_df=pd.read_csv('data/credits.csv',dtype='unicode')
  
  credit_df=convert_datatype_cast(credit_df)
  credit_df=credit_df[credit_df['id'].isin(current_ids)]
  credit_df.drop_duplicates(subset=['id'],inplace=True)
  credit_df = credit_df.reset_index(drop=True)
  #1.
  all_cast =get_all_cast_dictionary(credit_df,'cast')
  all_crew=get_all_cast_dictionary(credit_df,'crew')
  #2.
  top_n_list=keep_max_n_values(all_cast,9000)
  top_n_list2=keep_max_n_values(all_crew,9000)
  #3.
  count_column= get_counts_cast(credit_df,top_n_list,'cast')
  count_column2= get_counts_cast(credit_df,top_n_list2,'crew')

  credit_df.drop(["cast"],axis=1,inplace=True)
  credit_df.drop(["crew"],axis=1,inplace=True)

  credit_df['cast'] = count_column
  credit_df['crew'] = count_column2
  
  credit_df = credit_df.reset_index(drop=True)
  return credit_df

def merge_main_and_credits(main_df,credit_df):
  new_df=pd.merge(main_df, credit_df, on="id")
  return new_df

def process_credits(df):
  current_ids=df['id'].tolist()
  # print(current_ids)
  credit_df =  get_and_process_credits(current_ids)
  df = merge_main_and_credits(df,credit_df)
  return df

df = process_credits(df)
df

,budget,id,imdb_id,language,popularity,release_date,revenue,runtime,keyword,Day,...,Australia,India,Canada,United Kingdom,Other_Country,France,United States of America,Italy,cast,crew
0,30000000,862,tt0114709,1.0,21.946943,1995-10-30,373554033,81.0,7,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12,52
1,65000000,8844,tt0113497,1.0,17.015539,1995-12-15,262797249,104.0,2,4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12,14
2,16000000,31357,tt0114885,1.0,3.859495,1995-12-22,81452156,127.0,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9,7
3,60000000,949,tt0113277,1.0,17.924927,1995-12-15,187436818,170.0,17,4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,40,53
4,35000000,9091,tt0114576,1.0,5.231580,1995-12-22,64350171,106.0,3,4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,11000000,395834,tt5362988,1.0,40.796775,2017-08-03,184770205,111.0,10,3,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,9,10
5316,12000000,24049,tt0479751,0.0,1.323587,2007-06-14,19000000,185.0,0,3,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4,10
5317,800000,62757,tt0933361,1.0,0.903061,2006-11-23,1328612,100.0,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
5318,2000000,63281,tt1718881,1.0,0.121844,2010-09-30,1268793,107.0,0,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0


In [25]:
# Run this command to save the pre-processed file in your drive

df.to_csv('data/preprocessed.csv', index=False, encoding='utf-8-sig')